# Inteligencia Artificial
# Clase 33 - Detección de Objetos con YOLO

En esta notebook vamos a reentrenar YOLO v4 para personalizar los objetos que puede detectar. Trabajaremos con un dataset de imágenes de saxofones obtenido del [Open Image Dataset de Google](https://storage.googleapis.com/openimages/web/index.html), una inmensa base de datos *open source* con millones de imágenes anotadas de 600 categorías distintas. Veremos cómo usar la implementación original de Darknet para hacer *fine tuning* del modelo partiendo de los pesos preentrenados.

<img src="https://miro.medium.com/max/1396/1*Co8xD0IWPaBiWr-Xfu38dw.jpeg">

El código de esta notebook se basa mayormente en los ejemplos de [The AI Guy](https://github.com/theAIGuysCode), mencionados por Alexey Bochkovskiy en su [publicación de divulgación en Medium](https://medium.com/@alexeyab84/yolov4-the-most-accurate-real-time-neural-network-on-ms-coco-dataset-73adfd3602fe).


## Setup

Para poder utilizar la implementación de Darknet de YOLO v4, seguiremos los siguientes pasos:

1. Clonar el [repositorio oficial](https://github.com/AlexeyAB/darknet).
2. Ajustar el Makefile para habilitar OpenCV y el uso de la GPU.
3. Construir Darknet.
4. Descargar los pesos del modelo preentrenado en el dataset [COCO](https://cocodataset.org/#explore) (acrónimo de *Common Objects in Context*).

In [ ]:
# Clonamos el repositorio oficial
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Ajustamos el Makefile para habilitar OpenCV y el uso de la GPU
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Construimos Darknet
!make

In [ ]:
# Descargamos los pesos del modelo preentrenado en COCO
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Definimos una función utilitaria para mostrar las predicciones del modelo
def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# Hacemos una prueba para verificar que el setup se hizo correctamete
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/giraffe.jpg
imShow('predictions.jpg')

##  Entrenando YOLO v4 en un dataset *custom*

En lugar de utilizar el modelo preentrenado para hacer inferencia sobre nuevas imágenes, ahora nos vamos a concentrar en reentrenarlo para que aprenda a detectar objetos diferentes a los del dataset COCO.

Para poder desarrollar un detector personalizado con YOLO v4, vamos a necesitar:

*   Un dataset *custom* debidamente etiquetado.
*   Un nuevo archivo .cfg, que define la configuración del modelo y el entrenamiento.
*   Archivos obj.data y obj.names, que les especifican al modelo cómo leer el dataset y qué clases contiene.
*   Un archivo train.txt que contiene las rutas a las imágenes de entrenamiento (opcionalmente, podemos incluir un test.txt también).

### Obtención y preparación del dataset

En esta oportunidad, utilizaremos un dataset de imágenes de saxos descargado de [Open Imagen Dataset](https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F06ncr).

YOLO v4 requiere que las etiquetas de cada imagen se encuentren detalladas dentro de un archivo `.txt` ubicado en el mismo directorio y con el mismo nombre que la imagen. La estructura de las anotaciones tiene que ser la siguiente:

`<object-class> <x_center> <y_center> <width> <height>`,

donde:

- `<object-class>` es un número que codifica una clase (enteros que van de 0 a -(n_clases-1)-;
- `<x_center>`, `<y_center>`, `<width>` y `<height>` definen la ubicación y el tamaño del *bounding box*. Son valores relativos al ancho y alto de la imagen -valores float en el rango (0.0 to 1.0]-.

Para facilitar la obtención de las imágenes y la correcta preparación de las etiquetas correspondientes, utilizamos el [OIDv4_ToolKit](https://github.com/theAIGuysCode/OIDv4_ToolKit), una herramienta específicamente desarrollada para *fetchear* datasets de esta base de datos libre disponibilizada por Google.

**Nota**: en [este video](https://www.youtube.com/watch?v=_4A9inxGqRM) encontrarán un tutorial paso a paso sobre el uso del OIDv4_ToolKit.


In [ ]:
# Montamos el Google Drive para acceder facilmente a los archivos que necesitamos
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Creamos un symbolic link para poder llamar al path "/content/gdrive/My\ Drive/" como /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

In [ ]:
# El directorio /mydrive/yolov4 contiene los archivos que necesitamos 
!ls /mydrive/yolov4

In [ ]:
# Copiamos los zips con la data al entorno de trabajo de Colab
!cp /mydrive/yolov4/data/obj.zip /content/
!cp /mydrive/yolov4/data/test.zip /content/

In [ ]:
# Extraemos los archivos en la carpeta /darknet/data
%cd darknet
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [ ]:
import os
print('Hay {} imágenes en el set de train y {} en el set de test.'.format(len(os.listdir('./data/obj'))//2,
                                                                          len(os.listdir('./data/test'))//2))

### Configuración de la arquitectura: archivo `.cfg`

YOLO v4 requiere de un archivo con extensión `.cfg` que define la configuración de la arquitectura de la red y otras cuestiones importantes para el entrenamiento. Hasta el momento, estuvimos utilizando la configuración propia de `yolov4.cfg`, pero ahora es preciso hacer algunos cambios para adaptar al modelo a nuestro propio dataset.

Para agilizar el entrenamiento de nuestro detector de saxos, en lugar de usar el modelo completo, entrenaremos una versión más pequeña, llamada Tiny YOLO. Podemos tomar el molde de esta configuración del archivo `yolov4-tiny-custom.cfg` y adaptarlo siguiendo algunos [lineamientos sugeridos en el repositorio oficial de Darknet](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects):

- `batch` = 64
- `subdivisions` = 16
- `max_batches` = classes*2000 (pero no menos de 6000)
- `steps` = 0.8\*classes, 0.9\*classes
- `width` = 416 (o cualquier múltiplo de 32)
- `height` = 416 (o cualquier múltiplo de 32)
- `filters` = (classes + 5) * 3 en las tres capas convolucionales antes de las *layers* de YOLO.

Nuestro archivo `yolov4-tiny-obj.cfg` ya tiene las modificaciones pertinentes, por lo que simplemente lo depositamos en el directorio darknet/cfg:

In [ ]:
# Copiamos nuestro .cfg a la carpeta con los archivos de configuración
!cp /mydrive/yolov4/yolov4-tiny-obj.cfg ./cfg

### Archivos `obj.data` y `obj.names`

Estos archivos permiten indicarle al modelo cuántas clases hay en el dataset (y cuáles son), así como la ubicación de los archivos desde los que leerá la data y el path del directorio en el que irá guardando automáticamente los *checkpoints*.

Mientras que `obj.names` es simplemente una lista de los nombres de cada clase (una por línea en el archivo y respetando el orden de la codificación propia de las etiquetas), la estructura de `obj.data` debe ser la siguiente:

```
classes = # classes
train   = path/to/train.txt
valid   = path/to/test.txt
names   = path/to/obj.names
backup  = path/to/backup/
```

In [ ]:
# Copiamos los archivos  obj.data y obj.names a la carpeta /darknet/data
!cp /mydrive/yolov4/data/obj.names ./data
!cp /mydrive/yolov4/data/obj.data  ./data

### Archivos `train.txt` y `test.txt`

Por último, sólo nos queda generar los archivos `train.txt` y `test.txt`, que contienen las rutas de las imágenes de entrenamiento y testeo (una por línea). Para generarlos, usaremos los *scripts* utilitarios `generate_train.py` y `generate_test.py`, cortesía de [The AI Guy](https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/tree/master/yolov4):

In [ ]:
# Copiamos los programas
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
# Los ejecutamos...
!python generate_train.py
!python generate_test.py

In [ ]:
# ... y verificamos que se hayan generado los archivos train.txt y test.txt en darknet/data
!ls data/

¡Excelente! Ya estamos en condiciones de entrenar nuestro propio detector personalizado. En lugar de entrenarlo desde cero, como es habitual, aprovecharemos las virtudes del *transfer learning* y tomaremos los pesos preentrenados de las primeras 29 capas convolucionales como punto de partida:

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

¡Perfecto! Ahora sí, sólo nos queda comenzar a entrenar. El comando para hacerlo será

```
# En Linux
./darknet detector train <path to obj.data> <path to custom config> [<path to pre-trained weights>] [-dont_show] [-map]

# En WindowsLinux
darknet.exe detector train <path to obj.data> <path to custom config> [<path to pre-trained weights>] [-dont_show] [-map]
```

Aunque no es estrictamente necesario, el *flag* `-map` nos permite evaluar la métrica mean Average Precision durante el entrenamiento sobre el conjunto de datos de validación.

In [ ]:
# Entrenamos custom detector (descomentar %%capture si incurrimos en problemas de memoria o si Colab crashea)
#%%capture
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map

Como el entrenamiento es algo costoso, ya contamos con los pesos guardados cada 1000 iteraciones completas. Podemos comparar las métricas de mAP alcanzadas a lo largo del entrenamiento fácilmente:

In [ ]:
# El comando map permite evaluar esta métrica
!./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_1000.weights # Al cabo de 1000 iteraciones

In [ ]:
# El comando map permite evaluar esta métrica
!./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_3000.weights # Al cabo de 3000 iteraciones

In [ ]:
# El comando map permite evaluar esta métrica
!./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights # Al cabo de 6000 iteraciones

Vemos cómo nuestro modelo va mejorando hasta concluir su entrenamiento con un 84% de mAP. ¡Nada mal para un dataset de menos de 1000 imágenes!

## Probando nuestro detector sobre imágenes nuevas

Llego el momento de la verdad. Veamos cuán bien funciona nuestro detecto con imágenes nuevas:

In [ ]:
# Primero hacemos inferencia con el modelo preentrenado en COCO para comparar los resultados
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/yolov4/imgs/bird.jpg
imShow('predictions.jpg')

In [ ]:
# Ahora, corremos nuestro propio detector
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights /mydrive/yolov4/imgs/bird.jpg
imShow('predictions.jpg')

¡Fantástico! Veamos algunos ejemplos más...

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights /mydrive/yolov4/imgs/coltrane.jpg
imShow('predictions.jpg')

¡Parece que nuestro detector es un éxito!

Veamos ahora algunos ejemplos más difíciles de detectar correctamente:

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights /mydrive/yolov4/imgs/colley.jpg
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights /mydrive/yolov4/imgs/bird_draw.jpg
imShow('predictions.jpg')

En estos últimos casos, el modelo no parece estar prediciendo del todo bien... Veamos qué ocurre si reducimos el umbral de confianza:

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights /mydrive/yolov4/imgs/colley.jpg -thresh 0.1
imShow('predictions.jpg')

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4/backup/yolov4-tiny-obj_6000.weights /mydrive/yolov4/imgs/bird_draw.jpg -thresh 0.1
imShow('predictions.jpg')

Sencillamente genial.

## Conclusines

- Vimos cómo  utilizar podemos construir un detector de objetos personalizado reentrenando YOLO v4 sobre un dataset propio.
- Exploramos el Open Image Dataset, una gran fuente de imágenes ya etiquetadas y listas para alimentar algoritmos de detección.
- La herramienta OIDv4_ToolKit nos facilita la descarga y preparación del dataset.
- En caso de querer generar nuestras propias etiquetas, existen algunas aplicaciones que nos ayudan en la tarea, como [LabelImg](https://github.com/tzutalin/labelImg) o [OpenLabeling](https://github.com/Cartucho/OpenLabeling).